In [6]:
import os 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
load_dotenv()

True

In [3]:
model=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
from langchain_community.document_loaders import TextLoader


# Try utf-8 first, fall back to latin-1 with replacement to avoid UnicodeDecodeError
try:
    loader = TextLoader(file_path, encoding="utf-8")
    loaded_data = loader.load()
    print(f"Loaded {len(loaded_data)} documents using utf-8")
except UnicodeDecodeError as e:
    print(f"utf-8 UnicodeDecodeError: {e}; trying latin-1 with replacement")
    loader = TextLoader(file_path, encoding="latin-1", errors="replace")
    loaded_data = loader.load()
    print(f"Loaded {len(loaded_data)} documents using latin-1 (with replacement)")
except Exception as e:
    print(f"Error loading with utf-8: {e}; trying latin-1 with replacement")
    try:
        loader = TextLoader(file_path, encoding="latin-1", errors="replace")
        loaded_data = loader.load()
        print(f"Loaded {len(loaded_data)} documents using latin-1 (with replacement)")
    except Exception as e2:
        raise RuntimeError(f"Failed to load file: {e2}")

# Print a short preview of each loaded Document (page_content)
for i, doc in enumerate(loaded_data, 1):
    print(f"--- Document {i} ---")
    content = getattr(doc, "page_content", None) or getattr(doc, "content", None) or str(doc)
    print(content[:1000])
    print()


Loaded 1 documents using utf-8
--- Document 1 ---
Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.

Last year COVID-19 kept us apart. This year we are finally together again.

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.

With a duty to one another to the American people to the Constitution.

And with an unbending resolve that freedom will always triumph over tyranny.

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined.

He met the Ukrainian people.

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.

Groups of citizens 

In [54]:
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
chunks_of_text=text_splitter.split_documents(loaded_data)

In [55]:
len(chunks_of_text)

2

In [87]:

embeddings_model=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
vector_db=FAISS.from_documents(chunks_of_text,embedding=embeddings_model)

In [66]:
retriever=vector_db.as_retriever(kwargs={"k":2})

In [67]:
response=retriever.invoke("What did President Zelenskyy say in his speech to the European Parliament?")
print(response)

[Document(id='95dd9483-9cae-4303-9fad-b2f756a5b7df', metadata={'source': 'C:\\Users\\91800\\Desktop\\GENAI UDEMY\\txt files\\stateofunion.txt'}, page_content='Groups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland.\n\nIn this struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkness.” The Ukrainian Ambassador to the United States is here tonight.\n\nLet each of us if you’re able to stand and send an unmistakable signal to Ukraine and to the world.\n\nThank you. Thank you, thank you.\n\nShe’s bright, she’s strong, and she’s resolved.\n\nWe the People of the United States of America stand with the Ukrainian people.\n\nThroughout our history we’ve learned this lesson when dictators do not pay a price for their aggression they cause more chaos.\n\nThey keep moving.\n\nAnd the costs and the threats to America and the world keep rising.'), Document(id='eaf663ba

# Simple Use with LCEL

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [70]:
template="""Answer the question based only on the following context: 

{context}

Question:{question}
"""

prompt=ChatPromptTemplate.from_template(template=template)

model=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [81]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])



chain=(
    {"context":retriever | format_docs, "question": RunnablePassthrough()}|prompt|model|StrOutputParser()
)


In [83]:
response=chain.invoke("What did President Zelenskyy say in his speech to the European Parliament?")


In [86]:
response

'In his speech to the European Parliament, President Zelenskyy said: "Light will win over darkness."'